---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

text  target
0  Go until jurong point, crazy.. Available only ...       0
1                      Ok lar... Joking wif u oni...       0
2  Free entry in 2 a wkly comp to win FA Cup fina...       1
3  U dun say so early hor... U c already then say...       0
4  Nah I don't think he goes to usf, he lives aro...       0
5  FreeMsg Hey there darling it's been 3 week's n...       1
6  Even my brother is not like to speak with me. ...       0
7  As per your request 'Melle Melle (Oru Minnamin...       0
8  WINNER!! As a valued network customer you have...       1
9  Had your mobile 11 months or more? U R entitle...       1

In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)
print(X_train.shape)
print(X_test.shape)

(4179,)
(1393,)


### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    return (spam_data[spam_data["target"]==1].shape[0]/spam_data.shape[0])*100#Your answer here

In [4]:
answer_one()

13.406317300789663

In [5]:
"""
CountVectorizer 類會將文本中的詞語轉換為詞頻矩陣，
例如矩陣中包含一個元素a[i][j]，它表示j詞在i類文本下的詞頻。
它通過fit_transform 函數計算各個詞語出現的次數，
通過get_feature_names()可獲取詞袋中所有文本的關鍵字，
通過toarray()可看到詞頻矩陣的結果。
"""

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1)
corpus = ['This is the first document.','This is the second second document.','And the third one.','Is this the first document?']
X = vectorizer.fit_transform(corpus)
feature_name = vectorizer.get_feature_names()
#print (X)
print (feature_name)
print (X.toarray())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
[[0 1 1 1 0 0 1 0 1]
 [0 1 0 1 0 2 1 0 1]
 [1 0 0 0 1 0 1 1 0]
 [0 1 1 1 0 0 1 0 1]]


### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vect = CountVectorizer().fit(X_train)
    ans=vect.get_feature_names()
    new=sorted(ans ,reverse=True,key=lambda d: len(d))
    return new[0]

In [7]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    clf = MultinomialNB(alpha=0.1)
    clf.fit(X_train_vectorized,y_train)
    X_test_vect=vect.transform(X_test)
    predictions = clf.predict(X_test_vect)
    return  roc_auc_score(y_test, predictions)

In [9]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
"""
TF-IDF的主要思想是：如果某個詞或短語在一篇文章中出現的頻率TF高，
並且在其他文章中很少出現，則認為此詞或者短語具有很好的類別區分能力，適合用來分類。
某一特定文件內的高詞語頻率，以及該詞語在整個文件集合中的低文件頻率，可以產生出高權重的TF-IDF。
因此，TF-IDF傾向於過濾掉常見的詞語，保留重要的詞語。
"""
def answer_four():
    #max(0)返回該矩陣中每一行的最大值
    X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)
    vect = TfidfVectorizer().fit(X_train)
    feature_names = np.array(vect.get_feature_names())
    X_train_vectorized = vect.transform(X_train)
    sorted_tf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    smallest=list(zip(feature_names[sorted_tf_index[:20]],sorted(X_train_vectorized.max(0).toarray()[0])[:20]))
    biggest=list(zip(feature_names[sorted_tf_index[-20:]][::-1],sorted(X_train_vectorized.max(0).toarray()[0])[-20:][::-1]))
    smallest=pd.Series([features[1] for features in smallest],index=[features[0] for features in smallest])
    biggest=pd.Series([features[1] for features in biggest],index=[features[0] for features in biggest])
    return (smallest,biggest)

In [11]:
answer_four()#.shape

(sympathetic     0.074475
 healer          0.074475
 aaniye          0.074475
 dependable      0.074475
 companion       0.074475
 listener        0.074475
 athletic        0.074475
 exterminator    0.074475
 psychiatrist    0.074475
 pest            0.074475
 determined      0.074475
 chef            0.074475
 courageous      0.074475
 stylist         0.074475
 psychologist    0.074475
 organizer       0.074475
 pudunga         0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, 146tf150p    1.000000
 havent       1.000000
 home         1.000000
 okie         1.000000
 thanx        1.000000
 er           1.000000
 anything     1.000000
 lei          1.000000
 nite         1.000000
 yup          1.000000
 thank        1.000000
 ok           1.000000
 where        1.000000
 beerage      1.000000
 anytime      1.000000
 too          1.000000
 done         1.000000
 645          1.000000
 tick         0.980166
 blank        0.932702
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [12]:
def answer_five():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    clf = MultinomialNB(alpha=0.1)
    clf.fit(X_train_vectorized,y_train)
    X_test_vect=vect.transform(X_test)
    predictions = clf.predict(X_test_vect)
    return  roc_auc_score(y_test, predictions)

In [13]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [14]:
def answer_six():
 
    spam_data["number of characters"]=spam_data["text"].apply(lambda x:len(x))
    average_length_not_spam=np.mean(spam_data[spam_data["target"]==0]["number of characters"])
    average_length_spam=np.mean(spam_data[spam_data["target"]==1]["number of characters"])
    return (average_length_not_spam,average_length_spam)

In [15]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [16]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [17]:
from sklearn.svm import SVC

def answer_seven():
    X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized=vect.transform(X_test)
    clf = SVC(C=10000)
    X_train, X_test, y_train, y_test = train_test_split(spam_data['number of characters'], 
                                                    spam_data['target'], 
                                                    random_state=0)
    X_train_vectorized=add_feature(X_train_vectorized,X_train)

    clf.fit(X_train_vectorized,y_train)
    
    X_test_vectorized=add_feature(X_test_vectorized,X_test)
    predictions = clf.predict(X_test_vectorized)
    return  roc_auc_score(y_test, predictions)#Your answer here

In [18]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [19]:
def is_digit(x):
    ans=[]
    for i in range(len(x)):
        if x[i].isdigit():
            ans.append(x[i])
    return ans
def answer_eight():
    spam_data["new"]=spam_data["text"].apply(is_digit)
    spam_data["number of digits"]=spam_data["new"].apply(lambda x:len(x))
    average_length_not_spam=spam_data[spam_data["target"]==0]["number of digits"].mean()
    average_length_spam=spam_data[spam_data["target"]==1]["number of digits"].mean()
    return (average_length_not_spam,average_length_spam)  

In [20]:
answer_eight()

(0.29927461139896372, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [21]:
from sklearn.linear_model import LogisticRegression

def answer_nine(): 
    X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized=vect.transform(X_test)
    clf = LogisticRegression(C=100)
    X_train, X_test, y_train, y_test = train_test_split(spam_data['number of digits'], 
                                                    spam_data['target'], 
                                                    random_state=0)
    X_train_vectorized=add_feature(X_train_vectorized,X_train)

    clf.fit(X_train_vectorized,y_train)
    
    X_test_vectorized=add_feature(X_test_vectorized,X_test)
    predictions = clf.predict(X_test_vectorized)
    return  roc_auc_score(y_test, predictions)#Your answer here

In [22]:
answer_nine()

0.96787090640544626

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [23]:
import re
def answer_ten():
    pattern = '\W'
    spam_data["new_text"]=spam_data["text"].apply(lambda x:re.findall(pattern,x))
    spam_data["len"]=spam_data["new_text"].apply(lambda x:len(x))
    #better way
    #spam_data['length'] = spam_data['text'].str.findall(r'(\W)').str.len()
    average_length_not_spam=spam_data[spam_data["target"]==0]["len"].mean()
    average_length_spam=spam_data[spam_data["target"]==1]["len"].mean()
    return (average_length_not_spam,average_length_spam)

In [24]:
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [26]:
def answer_eleven():
    CV = CountVectorizer(min_df=5, ngram_range=(2,5),analyzer='char_wb')
    X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)
    vect = CV.fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized=vect.transform(X_test)
    clf = LogisticRegression(C=100)
    X_train, X_test, y_train, y_test = train_test_split(spam_data[['number of digits','len',"number of characters"]], 
                                                    spam_data['target'], 
                                                    random_state=0)
    X_train_vectorized=add_feature(X_train_vectorized,X_train.T)

    clf.fit(X_train_vectorized,y_train)
    X_test_vectorized=add_feature(X_test_vectorized,X_test.T)
    predictions = clf.predict(X_test_vectorized)
    feature_names = np.array(vect.get_feature_names()+['number of digits','len',"number of characters"])
    sorted_coef_index = clf.coef_[0].argsort()
    smallest=feature_names[sorted_coef_index[:10]]
    biggest=feature_names[sorted_coef_index[-10:]][::-1]
    
    return  (roc_auc_score(y_test, predictions),smallest,biggest)#Your answer here

In [27]:
answer_eleven()

(0.97885931107074342,
 array(['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'], 
       dtype='<U20'),
 array(['number of digits', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x',
        'ww', 'ar'], 
       dtype='<U20'))